# Learnings

- Welche Grafik für was?
    - siehe die zwei Bilder im Ordner"Hintergrundbilder"
    -  https://uen.pressbooks.pub/uvumqr/chapter/4-3-data-organization-and-representations/

# ???
- ??? Was ist der Unterschied zwischen import x as x unf from datetime import datetime
- ??? notebook 3: df = pd.read_csv("data/survey_results_public.csv", index_col = "Respondent") -> warum benötige ich index_col?
- ??? notebook 4: What is the difference between s = pd.Series([1, 2, 3, 4, 5]) and s = pd.DataFrame([1, 2, 3, 4, 5])
- ??? Wie kann ich dataframe einfach löschen, damit ich nicht jedes Mal das Notebook neustarten muss, wenn ich den df zerschossen habe?

# TODO
- TODO: requirements.txt erstellen und exportieren -> Ziel: Jemand soll den Code von GitHub runter laden sollen und das Projekt lokal zum laufen bringen können
- ~~TODO: Jeder Zelle kurz beschreiben (als Kommentat in ich-Perspektive)~~
- ~~TODO: Alles Header klein schreiben und mit "_" verbinden~~
- TODO: Am Anfang den dataframe verkleiner (z.B. nur auf Zeitraum von einem Jahr)
- ~~TODO: In Dataanalysis der dataframe möglichst unberührt lassen -> für jeden Schritt in der Datenanalyse einen eigenen temporären dataframe erstellen (wenn benötigt: in diesem dataframe auch die eigenen features engineeren)~~
- TODO: Um dataframe in einzelne Aktien zu zerlegen -> df.pivot() verwenden -> df.melt() macht dies wieder rückgängig
- TODO: dataframe "permanent" im Notebook hochladen, damit ich dan nicht jedes Mal erneut hochladen muss
- TODO: Später beschreiben wie der Pfad anzupassen ist, wenn die csv von wo anders hochgeladen werden soll
- TODO: Columns in features umbenennen

# Ideen für die Zukunft
- Modell für prediction von Aktienkursen erstellen
- poetry verwenden (für Verwaltung von dependencies)
- Präsentation mit Streamlit
- Pipeline erstellen
    - Daten werden automatisch (täglich/stündlich) gecrawlt
    - alte Daten werden verworfen, neue Daten werden ergänzt
    - Modell wird auf aktuellen Daten traineirt
    - deployment als webservice oder ähnliches

# 1. Setup

In [ ]:
# Here I install the needed packages (only execute once)

# install  packages
# !conda install jupyter
# !conda install numpy
# !conda isntall pandas

# TODO: Später hier alle packete installieren und importieren

In [ ]:
# Here I import the needed packages (only execute once)

# import packages
# !conda install jupyter
import numpy as np
import pandas as pd
from datetime import datetime # ??? Warum wird hier so und drunter und drüber anders importiert?
import seaborn as sns
import matplotlib.pyplot as plt

# TODO: Später hier alle packete installieren und importieren

In [ ]:
# This way I limit the number of rows and columns (e.g. when calling "df.head()") (only execute once)
pd.set_option("display.max_columns", 150)
pd.set_option("display.max_rows", 150)

In [ ]:
# This way I can define the appearence of graphics created with seaborn (only execute once)
sns.set_theme()

# 2. Exploratory Data Analysis (EDA)

### 2.1 Data Description (statistical and visual go get a first impression)

Description of given features:
- date (Date): The date of the stock price data.
- open (Open): The opening price of the stock on that date.
- high (High): The highest price the stock reached during the trading day.
- low (Low): The lowest price the stock reached during the trading day.
- close (Close): The closing price of the stock on that date.
- volume (Volume): The trading volume, i.e., the number of shares traded on that date.
- dividends (Dividends): Dividends paid on that date (if any).
- stock_splits (Stock Splits): Information about stock splits (if any).
- brand_name (Brand_Name): The name of the brand or company.
- ticker (Ticker): Ticker symbol for the stock.
- industry_tag (Industry_Tag): The industry category or sector to which the brand belongs.
- country (Country): The country where the brand is headquartered or primarily operates

Description of engineered features:
- TODO

In [ ]:
# Here I read the csv and save it as dataframe ("df") (only excute once)

df = pd.read_csv("data/World-Stock-Prices-Dataset.csv")

In [ ]:
df.info()

In [ ]:
df.head(20)

In [ ]:
df.tail(150)

In [ ]:
df.shape

In [ ]:
for name in df.columns:
    print(f"{name:<20} {df[name].nunique():>6}")

In [ ]:
df.describe()

In [ ]:
# Here I create a boxplot for each numeric feature 
numeric_features =  ["Open", "High", "Low", "Close", "Volume", "Dividends", "Stock Splits", "Capital Gains"]

for name in numeric_features:
    sns.boxplot(data=df, y=name)
    plt.show()

In [ ]:
# Here I create a boxplot for each ceatogoric feature (this cell fails after renaming the features in the next cells)
categoric_features = ["Brand_Name",	"Ticker",	"Industry_Tag",	"Country"]

for name in categoric_features:
    plt.figure(figsize=(16, 4))
    sns.histplot(data=df, x=name)
    plt.xticks(rotation=90)
    plt.show()

### 2.2 Data Cleaning

#### Are my cells uniform? 

I unifiy the names of all features (no capital letters, all words are connected by "_").

In [ ]:
# rename the features
df.columns = df.columns.str.lower().str.replace(r'\s+|[-\s]', '_', regex=True)

#### Are there duplicates? 

I remove rows which are duplicated and keep only one of the rows.

In [ ]:
# What is the format of the feature "date"?
df.loc[0:6,"date"]

In [ ]:
# Before I change the type of date from string to datetime, I check for the granularity of the dataset

df['date_for_check'] = df['date'].str[:10]
df['time_for_check'] = df['date'].str[11:]

grouped = df.groupby(["date_for_check", "brand_name"]).agg({"time_for_check": "count"})

# grouped_with_filter = grouped[grouped["time_for_check"] > 1]
# grouped_with_filter

filter = df.duplicated(subset=["date_for_check", "brand_name"], keep='first') # TODO: Haben wir in den Übungen irgendwie anders gelöst mit den Filtern damals

# I remove instances the appear multiple times
df.drop(index=df[filter].index, inplace=True)

# I remove column(s) that I don't need anymore
df = df.drop(columns=['date_for_check', 'time_for_check'])

#### Are there gaps in the time series? (TODO: Ist timeframe das richge "wording"?)

I don't fill the gaps and I dont delete the gaps as I want to maintain the original characteristics of the dataset and avoid introducing potential biases.

If gaps exist, I assume that it is due to the fact that stock exchanges are closed on weekends and and holidays. Other reasons could be possible as well.

In [ ]:
# Convert 'date' to datetime and extract only the date part
df['date_for_check'] = pd.to_datetime(df['date'].str[:10])

# Count occurrences of each date directly
date_counts = df['date_for_check'].value_counts().sort_index().reset_index()

# Rename columns
date_counts.columns = ['date', 'count']

# Create a complete date range and ensure all dates are accounted for in the final DataFrame
date_counts = date_counts.set_index('date').reindex(pd.date_range(date_counts['date'].min(), date_counts['date'].max()), fill_value=0).rename_axis('date').reset_index()

# I remove column(s) that I don't need anymore
df = df.drop(columns=['date_for_check'])

In [ ]:
date_counts = date_counts.loc[(date_counts['date'] >= "2020-01-01") & (date_counts['date'] <= "2020-03-01")]

plt.figure(figsize=(16, 4))
sns.histplot(data = date_counts, x="date" ,binwidth=1, weights="count")
plt.xticks(rotation=90)
plt.show()

#### Feature Engineering for Data Analysis

In [ ]:
# Transform data from string ti datetime
df['date'] = pd.to_datetime(df['date'].str[:10])


# add column for year, month and day
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"]= df["date"].dt.day

# Set data as index
df.set_index('date', inplace=True)

# reorder the features in a sensefull manner (first the dates, numrical data, finally categorical data)
df = df[["year", "month", "day", "open", "high", "low", "close", "volume", "dividends", "stock_splits", "capital_gains", "brand_name",	"ticker",	"industry_tag",	"country"]]

# delet features that I dont need
# df = df.drop(columns=['xyz', 'xyz']) #TODO: am Ende noch machen 

#### Are there NAs?

In [ ]:
df.isnull().sum()

### 2.3 Data Analysis (Kilian)

#### Individual stock assessment (of a randomly chosen stock)

In [ ]:
filt = (df["brand_name"] == "microsoft")

# APPLY FILTER HERE.
df_windows = df.loc[filt]

In [ ]:
# Plot erstellen
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_windows, x='date', y='open')

# Achsen und Titel anpassen (TODO: anpassen)
plt.xlabel('Datum')
plt.ylabel('Volumen')
plt.title('Aktienvolumen über Zeit')

# Plot anzeigen
plt.show()

In [ ]:
df_windows.reset_index(inplace=True)
df_melted = df_windows.melt(id_vars=['date'], value_vars=['close', 'open'], var_name='feature', value_name='value')
df_windows.set_index('date', inplace=True)

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_melted, x='date', y='value', hue='feature')

# Achsen und Titel anpassen
plt.xlabel('Datum')
plt.ylabel('Wert')
plt.title('Entwicklung der Aktienpreise (open, close) über die Zeit')

# Plot anzeigen
plt.show()

In [ ]:
df_windows.reset_index(inplace=True)
df_melted = df_windows.melt(id_vars=['date'], value_vars=['high', 'low'], var_name='feature', value_name='value')
df_windows.set_index('date', inplace=True)

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_melted, x='date', y='value', hue='feature')

# Achsen und Titel anpassen
plt.xlabel('Datum')
plt.ylabel('Wert')
plt.title('Entwicklung der Aktienpreise (low, high) über die Zeit')

# Plot anzeigen
plt.show()

#### Cumulative stock assessment

QUESTION 1: What is the accumulatie closing price by industry?

In [ ]:
# Average closing price by industry
industry_avg = df.groupby('industry_tag')['close'].mean().sort_values(ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x=industry_avg.values, y=industry_avg.index)
plt.title('Average Closing Price by Industry')
plt.xlabel('Average Closing Price')
plt.ylabel('Industry')
plt.grid(True)
plt.show()

QUESTION 2: What is the accumulatie closing price by industry over time?

In [ ]:
# Prepare the data
df.reset_index(inplace=True)
average_open_by_sector = df[['date', 'ticker', 'open', 'industry_tag']].groupby(['date', 'industry_tag'])[['open']].mean().reset_index()
df.set_index('date', inplace=True)

# Create the line plot
plt.figure(figsize=(10, 6))
sns.lineplot(data=average_open_by_sector, x='date', y='open', hue='industry_tag')

# modify the plot
plt.title('Average Open time series ($) per sector')
plt.xlabel('')
plt.ylabel('')
plt.legend(title='Industry Sector', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()

QUESTION 3: WHat is the average closing price by country?

In [ ]:
# Average closing price by country
country_avg = df.groupby('country')['close'].mean().sort_values(ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x=country_avg.values, y=country_avg.index)
plt.title('Average Closing Price by Country')
plt.xlabel('Average Closing Price')
plt.ylabel('Country')
plt.grid(True)
plt.show()

### 2.3 Data Analysis (Ascan)

### 2.3 Data Analysis (Anahita)

### 2.3 Data Analysis (Minh)

### 2.3 Data Analysis (Sayed)


### 2.3 Data Analysis (Shirose)

### 2.3 Data Analysis (Lars)

# 3. (Modeling and/or Prediction)